In [3]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd
import birch_bellwether

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [4]:
def cluster_driver(df,print_tree = False):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

In [9]:
#get cluster wise data for summarzation using median
def find_bellwether(data_source1,clusters,path,fold):
    df_train = pd.read_pickle(data_source1 + '/train_data.pkl')
    cluster,cluster_tree = build_BIRCH(df_train)
    cluster_ids = []
    cluster_structure = {}
    size = {}
    for key in cluster_tree:
        if cluster_tree[key].depth != None:
            cluster_ids.append(key)
            if cluster_tree[key].depth not in cluster_structure.keys():
                cluster_structure[cluster_tree[key].depth] = {}
            cluster_structure[cluster_tree[key].depth][key] = cluster_tree[key].parent_id
            size[key] = cluster_tree[key].size
    goals = ['f1','g','recall','precision','pf','pci_20','ifa']
    for _ in range(1):
        count = 0
        count_not = 0
        count_yes = 0
        score = []
        score_med = []
        cluster_info = {}
        for cluster in clusters:
            if cluster.rsplit('/',1)[1] in ['results','cdom_level1']:
                continue
            df = pd.read_csv(cluster + '/cdom_latest.csv')
            counts = {}
            med_count = []
            c_dom = df.wins.values.tolist()
            best_project = df.iloc[c_dom.index(max(c_dom)),0]
            for goal in goals:
                goal_df = pd.read_csv(cluster + '/1385_LR_bellwether_' + goal + '.csv')
                goal_df.rename(columns={'Unnamed: 0':'projects'},inplace=True)
                j = goal_df[goal_df['projects'] == best_project].values[0][1:]
                if goal == 'pci_20':
                    value = sum(i >= 0.40 for i in j)
                elif goal != 'pf':
                    value = sum(i >= 0.66 for i in j)
                else:
                    value = sum(i <= 0.33 for i in j)
                counts[goal] = value
            score_med.append([int(cluster.rsplit('/',1)[1]),
                              counts['f1'],
                              counts['g'],
                              counts['recall'],
                              counts['precision'],
                              counts['pf'],
                              counts['pci_20'],
                              max(c_dom),
                              best_project])
        score_df = pd.DataFrame(score_med, columns = ['id','count_f1',
                                                      'count_g','count_recall',
                                                      'count_precision','count_pf','count_pci_20',
                                                      'cdom_score','bellwether'])
        score_df = score_df.sort_values('id')
        score_df.to_csv(data_source1 + '/bellwether_cdom_2.csv')
        level_1_bellwethers = {}
        for cluster in cluster_structure[2].keys():
            if cluster_structure[2][cluster] not in level_1_bellwethers.keys():
                level_1_bellwethers[cluster_structure[2][cluster]] = []
            level_1_bellwethers[cluster_structure[2][cluster]].append(score_df[score_df['id'] == cluster].bellwether.values[0])
        score_med = []
        for key in  level_1_bellwethers.keys():
            sub_cluster_bellwethers = level_1_bellwethers[key]
            bell = birch_bellwether.bellwether(path,df_train)
            final_score = bell.bellwether(sub_cluster_bellwethers,sub_cluster_bellwethers)
            with open(data_source1 + '/cdom_level1/cluster_'  + str(key) + '_performance.pkl', 'wb') as handle:
                pickle.dump(final_score, handle, protocol=pickle.HIGHEST_PROTOCOL)             

In [10]:
for i in range(1):
    path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
    data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/2/fold_' + str(i)
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = data_source1 + '/'
    else:
        _dir = data_source1 + '\\'

    clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]
    find_bellwether(data_source1,clusters,path,i)

ossim.csv
zscreen.csv
jfreereport.csv
metacosm.csv
turbotrader-bos.csv
pfaedit.csv
aime.csv
pixel-commandos.csv
wow-qrsk.csv
google-caja.csv
lsb.csv
riff-evolve.csv
galaxium.csv
ftm.csv
kftpgrabber.csv
mule.csv
amygdala.csv
mp-rechnungs-und-kundenverwaltung.csv
reaper3d.csv
opennms.csv
retromenu.csv
tycho.csv
thrust.csv
bionote.csv
chipster.csv
zildo.csv
gwanted.csv
adobe-source.csv
piccolo2d.csv
haggle.csv
mevenide.csv
sabbu.csv
psrchive.csv
mptvseries.csv
nassp.csv
etics.csv
reaper-ecad.csv
genoviz.csv
jcae.csv
neodatis-odb.csv
moviesandbox.csv
xbplayer.csv
atomsite.csv
jsecurity.csv
xbg.csv
stemkit.csv
bibedt.csv
biodwh.csv
geogebra.csv
qtractor.csv
nodal.csv
